In [1]:
import rpy2
import sys
sys.path.append('/home/thies/repos/BIU/') # Modify this to where you have downloaded the BIU library
import biu
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
plt.rcParams['svg.fonttype'] = 'none'

import seaborn as sns
import scipy

R = biu.R()
from rdmpy import RDM
RDM.meta(source="repos/LunaQuestionnaire/clean.ipynb")

# Load the Isala Flow 1 questionnaire data

In [5]:
with RDM('../data/processed/merged_questionnaires.pkl', 'r', nouser=True) as ifd:
    Qraw = pd.read_pickle(ifd)

# Prepare the cleaned dataframe

In [14]:
QC = Qraw[['Q0.Deelnemersnummer']].rename(columns={'Q0.Deelnemersnummer':'IsalaID'}).copy()

# Clean questions

## Menstrual Cycle 

In [33]:
def determine_phases(row):
    """
    returns
     string
    """
    cycle_disrupted = row['Q1.Q52.3.[Hoeveel dagen zijn er gemiddeld tussen de eerste dag van je menstruatie\nen de eerste dag van je volgende? Je hoeft enkel het aantal dagen in te vullen indien je geen hormonale anticonceptie gebruikt die je cyclus kan beïnvloeden. - Selected Choice]'] == "Ik gebruik hormonale anticonceptie of een andere hormonale behandeling die mijn menstruele cyclus kan beïnvloeden"
    cycle_days     = row['Q1.Q52.3_4_TEXT.[Hoeveel dagen zijn er gemiddeld tussen de eerste dag van je menstruatie\nen de eerste dag van je volgende? Je hoeft enkel het aantal dagen in te vullen indien je geen hormonale anticonceptie gebruikt die je cyclus kan beïnvloeden. - Ik gebruik geen hormonale anticonceptie of een andere hormonale behandeling die mijn menstruele cyclus kan beïnvloeden - Tekst]']
    current_day    = row['A.day_of_cycle']
    
    if cycle_disrupted:
        return 'not_applicable'
    #fi
    
    if isinstance(cycle_days, str):
        return "unknown"
    #fi
    
    if pd.isna(cycle_days):
        cycle_days = 28
    #fi
    if pd.isna(current_day):
        return 'unknown'
    #fi
    
    if (cycle_days < 21) or (cycle_days > 35):
        return 'unknown'
    #fi
    
    if cycle_days <= 24:
        ovulation = cycle_days - 12
    elif cycle_days <= 31:
        ovulation = cycle_days - 14
    else:
        ovulation = cycle_days - 16
    #fi
    
    if current_day <= (ovulation-6):
        return 'Follicular'
    elif current_day <= (ovulation+2):
        return 'Ovulation'
    else:
        return 'Luteal'
    #fi
    
    return 'unknown'
#edef

phase = Qraw.apply(determine_phases, axis=1)



col_values = ['Luteal', 'Ovulation', 'Follicular']
for cv in col_values:
    QC['Reproductive.Cycle.Phase.%s' % cv] = phase.apply(lambda x: True if x == cv else None if x not in col_values else False)
#efor

QC['Reproductive.PeriMenopause'] = Qraw['Q1.Q52.2.[Heeft dit te maken met één van de onderstaande mogelijkheden? - Selected Choice]'].apply(lambda x: str(x).lower() in ['c','d'])


## Partner gender

In [30]:
col = 'Q1.Q4.[Mijn partner(s) is/zijn...]'
biu.ops.lst.freq(Qraw[col])
col_values = {'Man' : 'Man', 'Vrouw' : 'Woman'}
for cv in col_values:
    QC['Lifestyle.Partner.Is%s' % col_values[cv]] = Qraw[col].apply(lambda x: True if x == cv else None if x not in col_values else False)
    print('Lifestyle.Partner.Is%s' % col_values[cv])
#efor

Lifestyle.Partner.IsMan
Lifestyle.Partner.IsWoman


## Miscarriage, abortions

In [37]:
_rel = Qraw[[c for c in Qraw if 'uitgedragen' in c]]

for rtype, rvalues in {'Miscarriage': ['Nee afgebroken, miskraam met medicatie', 'Nee afgebroken, spontane miskraam', 'Nee afgebroken, curettage miskraam'],
                      'PretermBirth': ['Nee, vroeggeboorte'],
                      'Abortion': ['Nee afgebroken, abortus']}.items():
    QC['Reproductive.Pregnancy.%s' % rtype] = _rel.applymap(lambda x: x in rvalues).any(axis=1)
#efor

## Some previous questions

In [18]:
QC['Lifestyle.Drugs.Current_smoker'] = Qraw['Q1.Q21.[Rook je? Geef ook op hoeveel sigaretten je per dag rook(te) - Selected Choice]'].apply(
    lambda x: None if pd.isna(x) else x == 'Ja')

QC['Lifestyle.Drugs.Current_druguser'] = Qraw['Q1.Q22.[Gebruik je drugs (exclusief alcohol)? Gelieve ook aan te geven welke. Deze informatie wordt pseudoniem verwerkt. Wij hebben het grootste respect voor jou en voor je privacy. - Selected Choice]'].apply(
    lambda x: None if pd.isna(x) else x == 'Ja')

QC['Reproductive.Pregnancy.Attempted'] = Qraw['Q1.Q49.[Heb je ooit actief geprobeerd om zwanger te worden?]'].apply(
    lambda x: None if pd.isna(x) else x == 'Ja')

QC['Reproductive.Breastfeeding'] = Qraw['Q2.Q24.[Geef je op dit moment borstvoeding?]'].apply(
    lambda x: None if pd.isna(x) else x == 'Ja')

In [44]:
QC['Nutrition.EatsMeat'] = Qraw['Q1.Q29_4.[Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Vlees]'].apply(
    lambda x: None if pd.isna(x) else x not in ['Zelden', 'Nooit'])

QC['Nutrition.EatsFish'] = Qraw['Q1.Q29_6.[Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Vis]'].apply(
    lambda x: None if pd.isna(x) else x in ['Zelden', 'Nooit'])

QC['Nutrition.EatsAnimals'] = Qraw['Q1.Q29_5.[Hoe regelmatig heb je de onderstaande zaken gegeten of gedronken in de afgelopen 3 maanden? - Dierlijke producten (eieren, kaas, gelatine,...)]'].apply(
    lambda x: None if pd.isna(x) else x in ['Zelden', 'Nooit'])

QC['Nutrition.Vgetarian'] = QC.apply(
    lambda x: None if any(pd.isna(x[['Nutrition.EatsMeat','Nutrition.EatsFish']])) else not(x['Nutrition.EatsFish'] or x['Nutrition.EatsMeat']), axis=1)

QC['Nutrition.Pescetarian'] = QC.apply(
    lambda x: None if any(pd.isna(x[['Nutrition.EatsMeat','Nutrition.EatsFish']])) else x['Nutrition.EatsFish'] and not(x['Nutrition.EatsMeat']), axis=1)

QC['Nutrition.Vegan'] = QC.apply(
    lambda x: None if any(pd.isna(x[['Nutrition.EatsMeat','Nutrition.EatsFish', 'Nutrition.EatsAnimals']])) else not(x['Nutrition.EatsFish'] or x['Nutrition.EatsMeat'] or x['Nutrition.EatsAnimals']), axis=1)


QC['Environment.Urban'] = Qraw['Q1.Q15.[Hoe zou je het gebied waar je nu woont omschrijven?]'].apply(
    lambda x: None if pd.isna(x) else x in ['Stadskern', 'Dorpskern', 'Residentiële woonwijk', 'Drukke baan', 'Industriezone'])

QC['Environment.Coutryside'] = Qraw['Q1.Q15.[Hoe zou je het gebied waar je nu woont omschrijven?]'].apply(
    lambda x: None if pd.isna(x) else x in ['Groene zone/ recreatiegebied/...', 'Landelijk gebied'])



In [47]:
def is_belgium(v):
    if pd.isna(v):
        return None
    #fi
    v = v.lower()
    if 'belg' in v:
        return True
    elif v in ['be', 'antwerpen', 'neen', '0', 'geen', 'b', 'beglie' ]:
        return True
    else:
        return False
    #fi
#edef

QC['Environment.Belgium.First3Years'] = Qraw['Q1.Q9.[In welk(e) land(en) heb je gewoond tot en met je derde levensjaar? Indien dit enkel België is, mag je het ook opgeven.]'].apply(
    lambda x: is_belgium(x))

QC['Lifestyle.Partner.OneVsSingle'] = Qraw['Q1.Q47.[Heb je de afgelopen drie maanden één of meerdere seksuele\npartner(s) gehad? - Selected Choice]'].apply(
    lambda x: None if (pd.isna(x) or x in ['Ja, meerdere vaste partners: hoeveel?', 'Ja, wisselende partners: hoeveel?']) else x == 'Ja, één vaste partner')
QC['Lifestyle.Partner.OneVsMany'] = Qraw['Q1.Q47.[Heb je de afgelopen drie maanden één of meerdere seksuele\npartner(s) gehad? - Selected Choice]'].apply(
    lambda x: None if (pd.isna(x) or x in ['Nee']) else x != 'Ja, één vaste partner')

QC['Environment.Economic.HouseholdContribution'] = Qraw['Q2.Q7.[Hoe groot is je aandeel aan het gezinsinkomen?]'].apply(
    lambda x: None if pd.isna(x) else {'Ik heb het grootste aandeel ': 3/4,
 'Ik draag als enige bij ': 4/4,
 'We dragen evenveel bij  ': 2/4,
 'Ik draag het minste bij  ': 1/4,
 'Ik draag niet bij  ': 0/4}.get(x, None))

QC['Environment.Economic.FinancialDifficulties'] = Qraw['Q2.Q4.[Kan je maandelijks rondkomen met de som van totaal beschikbaar gezinsinkomen (alles inbegrepen*)? \n\n\n\n* De leefomstandigheden van een huishouden hangen natuurlijk in belangrijke mate af van het inkomen. Het gaat hier over het totale beschikbare inkomen van de verschillende leden van het huishouden samen. Meerdere leden kunnen dus bijdragen. Het totale beschikbare inkomen van een huishouden bestaat uit, (1) netto-lonen, wedden en nettobedrijfsinkomens voor zelfstandigen, (2) sociale uitkeringen en kinderbijslagen, (3) bijkomende inkomens zoals huuropbrengsten, intresten e.a. De som van al deze inkomens voor alle personen uit je huishouden is het totale beschikbare inkomen van je huishouden.]'].apply(
    lambda x: None if pd.isna(x) else (x in ['Eerder moeilijk', 'Moeilijk', 'Zeer moeilijk']))

## Your additional questions

# Export cleaned questionnaire

In [49]:
with RDM('../data/cleaned/IsalaFlow1.CleanedQuestionnaires.xlsx', 'w', nouser=True) as ofd:
    QC.to_excel(ofd)
#ewith

with RDM('../data/cleaned/IsalaFlow1.CleanedQuestionnaires.xlsx', 'w', nouser=True) as ofd:
    QC.to_pickle(ofd)
#ewith